In [1]:
import pandas as pd
import wandb
import json
from pathlib import Path

In [2]:
method="procrustes" 
extend_exp=""
save_path = Path("./figures")
save_path.mkdir(exist_ok=True, parents=True)
vm_name = {
    'resnet152': 'ResNet152',
    'segformer-b5-finetuned-ade-640-640': 'SegFormer-B5',
    'vit-mae-huge': 'MAE-Huge',
}


In [3]:
api = wandb.Api()
df = pd.DataFrame()

model_name = {'bert_uncased_L-2_H-128_A-2': "BERT_TINY",
                'bert_uncased_L-4_H-256_A-4': "BERT_MINI" ,
                'bert_uncased_L-4_H-512_A-8': "BERT_SMALL",
                'bert_uncased_L-8_H-512_A-8': "BERT_MEDIUM",
                'bert-base-uncased': "BERT_BASE",
                'bert-large-uncased': "BERT_LARGE",
                'gpt2': "GPT2_BASE",
                'gpt2-medium': "GPT2_MEDIUM",
                'gpt2-large': "GPT2_LARGE",
                'gpt2-xl': "GPT2_XL",
                'opt-125m': "OPT_125M",
                'opt-1.3b': "OPT_1.3B",
                'opt-6.7b': "OPT_6.7B",
                'opt-30b': "OPT_30B",
                'opt-66b': "OPT_66B",
                'Llama-2-7b-hf': "Llama-2-7b",
                'Llama-2-13b-hf': "Llama-2-13b",
                'Llama-2-70b-hf': "Llama-2-70b",
                "fasttext":"fastText"}

model_size = {'bert_uncased_L-2_H-128_A-2':4.4, 'bert_uncased_L-4_H-256_A-4':11.3, 'bert_uncased_L-4_H-512_A-8':29.1, 
    'bert_uncased_L-8_H-512_A-8':41.7, 'bert-base-uncased':110, 'bert-large-uncased':340, 
    'gpt2':117,  'gpt2-large':762, 'gpt2-xl':1542, 'opt-125m':125, 'opt-6.7b':6700, 'opt-30b':30000,'opt-66b':66000,
    'Llama-2-7b-hf': 7000,
    'Llama-2-13b-hf': 13000,
    'Llama-2-70b-hf': 70000,}

model_tags = {
    "ft": "fasttext",
    "bert": "bert",
    "gpt2": "gpt2",
    "opt": "opt"
}

set_name = "cleaned"


# for model_alias in ["LM"]:
    # method_name = "Procrustes Analysis" if method == "procrustes" else "Ridge Regression"
runs = api.runs(path=f"jiaangli00/train_ratio_TACL")
    # metrics = runs.summary["Results"]

    # data = []
for i, single_run in enumerate(runs):
    metric = json.load(single_run.file(single_run.summary["Results"]["path"]).download(exist_ok=True))
    df = pd.concat([df,pd.DataFrame(metric["data"], columns=metric["columns"])])

# # df['Subjects'] = df['Subjects'].replace({f'brain_{i}': f'Subject-{i}' for i in range(1, num_subs + 1)})
# df['Layers'] = df['Layers'].replace({f'layer_{i}': f'layer-{i})
df['Models_size'] = df['LM'].copy()
df['Models_size'] = df['Models_size'].replace(model_size)
df['VM'] = df['VM'].replace(vm_name)
df['LM'] = df['LM'].replace(model_name)

# df = df.loc[(df["LM"] != "Llama-2-70b-hf") | (df["LM"] != "opt-66b")]
df = df[~df["LM"].isin(["Llama-2-70b","OPT_66B"])]
# #
group_names: list[str] = ["LM", "VM", "train_ratio"]
if extend_exp != "":
    group_names.append("Bins")
#
precision_csls, precision_nn = [], []
for k in [10, 100]:
    precision_csls.append(f'P@{k}-CSLS')
    precision_nn.append(f'P@{k}-NN')

precision = precision_csls
#
precision.append("Models_size")

In [4]:

seed_avg = df.groupby(group_names)[precision].mean().reset_index().sort_values(by='Models_size', ascending=True)
# seed_avg = df.groupby(group_names)[precision].mean().reset_index().round(1)

vm_selected = seed_avg.loc[(seed_avg["VM"] == "ResNet18") | (seed_avg["VM"] == "ResNet50") | (seed_avg["VM"] == "ResNet152") | (seed_avg["VM"] == "SegFormer-B0") | (seed_avg["VM"] == "SegFormer-B3") | (seed_avg["VM"] == "SegFormer-B5") | (seed_avg["VM"] == "MAE-Base") | (seed_avg["VM"] == "MAE-Large") | (seed_avg["VM"] == "MAE-Huge")]
# seed_avg["VM"][:14]
# vm_list = ["SegFormer-B0", "SegFormer-B3" , "SegFormer-B5", "MAE-Base" , "MAE-Large", "MAE-Huge", "ResNet18", "ResNet50" , "ResNet152"]
# vm_left = ["SegFormer-B1", "SegFormer-B2" , "SegFormer-B4","ResNet34", "ResNet101"]
vm_selected_left = seed_avg.loc[(seed_avg["VM"] == "ResNet34") | (seed_avg["VM"] == "ResNet101") | (seed_avg["VM"] == "SegFormer-B1") | (seed_avg["VM"] == "SegFormer-B2") | (seed_avg["VM"] == "SegFormer-B4")]


In [5]:
vm_selected.loc[vm_selected["VM"]=="MAE-Huge"].sort_values(by='train_ratio', ascending=True)

,LM,VM,train_ratio,P@10-CSLS,P@100-CSLS,Models_size
0,OPT_30B,MAE-Huge,0.1,0.648,5.254,30000.0
1,OPT_30B,MAE-Huge,0.5,2.634,15.306,30000.0
2,OPT_30B,MAE-Huge,1.0,4.626,23.654,30000.0
3,OPT_30B,MAE-Huge,5.0,13.857,46.488,30000.0
4,OPT_30B,MAE-Huge,10.0,18.648,53.659,30000.0
5,OPT_30B,MAE-Huge,15.0,21.224,56.818,30000.0
6,OPT_30B,MAE-Huge,20.0,23.042,58.240,30000.0
7,OPT_30B,MAE-Huge,25.0,24.552,59.680,30000.0
8,OPT_30B,MAE-Huge,30.0,25.554,60.712,30000.0
9,OPT_30B,MAE-Huge,40.0,27.158,62.614,30000.0


In [6]:
vm_test = vm_selected
vm_test

,LM,VM,train_ratio,P@10-CSLS,P@100-CSLS,Models_size
0,OPT_30B,MAE-Huge,0.1,0.648,5.254,30000.0
21,OPT_30B,ResNet152,30.0,32.038,70.488,30000.0
22,OPT_30B,ResNet152,40.0,33.766,71.752,30000.0
23,OPT_30B,ResNet152,50.0,34.566,72.384,30000.0
24,OPT_30B,ResNet152,60.0,35.446,72.906,30000.0
25,OPT_30B,ResNet152,70.0,35.962,73.154,30000.0
26,OPT_30B,SegFormer-B5,0.1,0.820,5.966,30000.0
27,OPT_30B,SegFormer-B5,0.5,2.722,16.730,30000.0
20,OPT_30B,ResNet152,25.0,31.230,69.590,30000.0
28,OPT_30B,SegFormer-B5,1.0,5.228,26.528,30000.0


In [28]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Define line styles
line_styles = ['dot', 'dashdot', 'longdash']

# Define marker symbols
marker_symbols = ['triangle-up', 'circle']
marker_size = 20
width_size = 5

# colors = ["#7fcdbb", "#1d91c0", "#253494"]
colors = px.colors.qualitative.Set2

# Create a single plot
fig = go.Figure()

mae_opt = vm_selected.loc[(vm_selected["VM"] == "MAE-Huge") & (vm_selected["LM"].str.startswith("OPT"))].sort_values(by='train_ratio', ascending=True)
res_opt = vm_selected.loc[(vm_selected["VM"] == "ResNet152") & (vm_selected["LM"].str.startswith("OPT"))].sort_values(by='train_ratio', ascending=True)
seg_opt = vm_selected.loc[(vm_selected["VM"] == "SegFormer-B5") & (vm_selected["LM"].str.startswith("OPT"))].sort_values(by='train_ratio', ascending=True)

precision_mae = np.array([mae_opt["P@10-CSLS"], mae_opt["P@100-CSLS"]]).T
precision_res = np.array([res_opt["P@10-CSLS"], res_opt["P@100-CSLS"]]).T
precision_seg = np.array([seg_opt["P@10-CSLS"], seg_opt["P@100-CSLS"]]).T

for idx, k in enumerate([10, 100]):
    line_style = line_styles[idx]
    marker_symbol = marker_symbols[idx]

    line1 = go.Scatter(x=mae_opt["train_ratio"], y=precision_mae[:, idx],
                      mode='lines+markers', line=dict(color=colors[0], dash=line_style, width=width_size),
                      marker_symbol=marker_symbol, legendgroup='MAE', name=f'MAE - K={k}',
                      marker=dict(size=marker_size))

    line2 = go.Scatter(x=res_opt["train_ratio"], y=precision_res[:, idx],
                      mode='lines+markers', line=dict(color=colors[1], dash=line_style, width=width_size),
                      marker_symbol=marker_symbol, legendgroup='ResNet', name=f'ResNet - K={k}',
                      marker=dict(size=marker_size))

    line3 = go.Scatter(x=seg_opt["train_ratio"], y=precision_seg[:, idx],
                      mode='lines+markers', line=dict(color=colors[2], dash=line_style, width=width_size),
                      marker_symbol=marker_symbol, legendgroup='SegFormer', name=f'SegFormer - K={k}',
                      marker=dict(size=marker_size))

    fig.add_trace(line1)
    fig.add_trace(line2)
    fig.add_trace(line3)

# fig.update_xaxes(range=[-1, 2], type="log")
fig.update_xaxes(range=[0, 75])
fig.update_yaxes(range=[0, 80], tickvals=[0, 10, 20, 30, 40, 50, 60, 70, 80])
fig.update_layout(
    xaxis=dict(
        tickfont=dict(size=23)
    ),
    yaxis=dict(
        tickfont=dict(size=23)
    ),
    # title=dict(text="Effect of Variation in Training Set Sizes", font=dict(size=30)),
    xaxis_title=dict(text="Training Ratio", font=dict(size=30)),
    yaxis_title=dict(text="Precision@K", font=dict(size=30)),
    legend=dict(x=0.25, y=-0.1, orientation="h", font=dict(size=20)),
    width=1200,
    height=1000
)

# Show the plot
fig.show()
fig.write_image(save_path / f"train_ratio_v2.pdf")
